In [29]:
import pathlib
import numpy as np
import pandas as pd
import json
from scipy.signal import butter, filtfilt

import plotly.express as px
import plotly.graph_objects as go


In [ ]:
def calculate_angle(a, b, c):
    """Returns the angle at point b formed by points a-b-c (in degrees)."""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

def extract_leg_joint_angles(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    frames = data["instance_info"]
    n_frames = len(frames)

    # Preallocate arrays
    angles = {
        'L_Shoulder_Pos':np.full((n_frames,2), np.nan),
        'L_Hip_Pos':np.full((n_frames,2), np.nan),
        'L_Knee_Pos':np.full((n_frames,2), np.nan),
        'L_Ankle_Pos':np.full((n_frames,2), np.nan),
        'L_1stToe_Pos':np.full((n_frames,2), np.nan),
        'L_5thToe_Pos':np.full((n_frames,2), np.nan),
        'L_Heel_Pos':np.full((n_frames,2), np.nan),

        'R_Shoulder_Pos':np.full((n_frames,2), np.nan),
        'R_Hip_Pos':np.full((n_frames,2), np.nan),
        'R_Knee_Pos':np.full((n_frames,2), np.nan),
        'R_Ankle_Pos':np.full((n_frames,2), np.nan),
        'R_1stToe_Pos':np.full((n_frames,2), np.nan),
        'R_5thToe_Pos':np.full((n_frames,2), np.nan),
        'R_Heel_Pos':np.full((n_frames,2), np.nan),

        'L_Hip_Angle': np.full(n_frames, np.nan),
        'L_Knee_Angle': np.full(n_frames, np.nan),
        'L_Ankle_Angle': np.full(n_frames, np.nan),

        'R_Hip_Angle': np.full(n_frames, np.nan),
        'R_Knee_Angle': np.full(n_frames, np.nan),
        'R_Ankle_Angle': np.full(n_frames, np.nan)
    }

    for i, frame in enumerate(frames):
        keypoints = frame['instances'][0]['keypoints']
        # Left side
        angles['L_Shoulder_Pos'][i] = keypoints[5]
        angles['L_Hip_Pos'][i] = keypoints[11]
        angles['L_Knee_Pos'][i] = keypoints[13]
        angles['L_Ankle_Pos'][i] = keypoints[15]
        angles['L_1stToe_Pos'][i] = keypoints[17]
        angles['L_5thToe_Pos'][i] = keypoints[18]
        angles['L_Heel_Pos'][i] = keypoints[19]

        # Right side
        angles['R_Shoulder_Pos'][i] = keypoints[6]
        angles['R_Hip_Pos'][i] = keypoints[12]
        angles['R_Knee_Pos'][i] = keypoints[14]
        angles['R_Ankle_Pos'][i] = keypoints[16]
        angles['R_1stToe_Pos'][i] = keypoints[20]
        angles['R_5thToe_Pos'][i] = keypoints[21]
        angles['R_Heel_Pos'][i] = keypoints[22]
    
        # except Exception:
        #     # If bad frame (e.g., divide by 0 or missing keypoints), keep NaN
        #     pass

        try:
            angles['L_Hip_Angle'][i] = calculate_angle(l_shoulder, l_hip, l_knee)
            angles['L_Knee_Angle'][i] = calculate_angle(l_hip, l_knee, l_ankle)
            angles['L_Ankle_Angle'][i] = calculate_angle(l_knee, l_ankle, l_big_toe)

            angles['R_Hip_Angle'][i] = calculate_angle(r_shoulder, r_hip, r_knee)
            angles['R_Knee_Angle'][i] = calculate_angle(r_hip, r_knee, r_ankle)
            angles['R_Ankle_Angle'][i] = calculate_angle(r_knee, r_ankle, r_foot)
        except Exception:
            # If bad frame (e.g., divide by 0 or missing keypoints), keep NaN
            pass
    return angles

In [44]:


# Define keypoint indices to filter (COCO WholeBody)
KEYPOINTS_TO_FILTER = {
    5: "left_shoulder", 6: "right_shoulder",
    7: "left_elbow", 8: "right_elbow",
    9: "left_wrist", 10: "right_wrist",
    11: "left_hip", 12: "right_hip",
    13: "left_knee", 14: "right_knee",
    15: "left_ankle", 16: "right_ankle",
    17: "left_big_toe", 18: "left_small_toe", 19: "left_heel",
    20: "right_big_toe", 21: "right_small_toe", 22: "right_heel"
}

def butterworth_filter(data, cutoff=15, fs=240, order=4):
    nyq = 0.5 * fs
    norm_cutoff = cutoff / nyq
    b, a = butter(order, norm_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=0)

def filter_keypoints(json_path,KEYPOINTS_TO_FILTER=KEYPOINTS_TO_FILTER,output_path=None):
    with open(json_path, 'r') as f:
        data = json.load(f)

    frames = data["instance_info"]
    n_frames = len(frames)

    # Preallocate arrays
    data = {
        'raw':{
            'L_Shoulder_Pos':np.full((n_frames,2), np.nan),
            'L_Hip_Pos':np.full((n_frames,2), np.nan),
            'L_Knee_Pos':np.full((n_frames,2), np.nan),
            'L_Ankle_Pos':np.full((n_frames,2), np.nan),
            'L_1stToe_Pos':np.full((n_frames,2), np.nan),
            'L_5thToe_Pos':np.full((n_frames,2), np.nan),
            'L_Heel_Pos':np.full((n_frames,2), np.nan),
            # Right Side Positions
            'R_Shoulder_Pos':np.full((n_frames,2), np.nan),
            'R_Hip_Pos':np.full((n_frames,2), np.nan),
            'R_Knee_Pos':np.full((n_frames,2), np.nan),
            'R_Ankle_Pos':np.full((n_frames,2), np.nan),
            'R_1stToe_Pos':np.full((n_frames,2), np.nan),
            'R_5thToe_Pos':np.full((n_frames,2), np.nan),
            'R_Heel_Pos':np.full((n_frames,2), np.nan),
        }
        
    }

    for i, frame in enumerate(frames):
        keypoints = frame['instances'][0]['keypoints']
        #Left Side
        data['raw']['L_Shoulder_Pos'][i] = keypoints[5]
        data['raw']['L_Hip_Pos'][i] = keypoints[11]
        data['raw']['L_Knee_Pos'][i] = keypoints[13]
        data['raw']['L_Ankle_Pos'][i] = keypoints[15]
        data['raw']['L_1stToe_Pos'][i] = keypoints[17]
        data['raw']['L_5thToe_Pos'][i] = keypoints[18]
        data['raw']['L_Heel_Pos'][i] = keypoints[19]
        # Right side
        data['raw']['R_Shoulder_Pos'][i] = keypoints[6]
        data['raw']['R_Hip_Pos'][i] = keypoints[12]
        data['raw']['R_Knee_Pos'][i] = keypoints[14]
        data['raw']['R_Ankle_Pos'][i] = keypoints[16]
        data['raw']['R_1stToe_Pos'][i] = keypoints[20]
        data['raw']['R_5thToe_Pos'][i] = keypoints[21]
        data['raw']['R_Heel_Pos'][i] = keypoints[22]

    # Apply filtering to selected keypoints
    data['filtered'] = {}
    for key,val in data['raw'].items():
        data['filtered'][key] = np.column_stack((butterworth_filter(val[:,0]), butterworth_filter(val[:,1])))

    return data

In [45]:
results_path = r"C:\Users\dgaytanj\mmpose\vis_results\results_sprint_start_sample.json"

In [46]:
filtered_data = filter_keypoints(results_path)


In [47]:
angles = extract_leg_joint_angles(results_path)
print(angles['R_1stToe_Pos'].shape)  # (n_frames,)
print(angles['R_1stToe_Pos']) 

(616, 2)
[[400.19866943 798.18634033]
 [403.87683105 798.28338623]
 [400.38867188 798.19830322]
 ...
 [532.42120361 731.26940918]
 [532.0680542  731.51141357]
 [531.77392578 731.56878662]]


In [48]:
angles['R_1stToe_Pos']

array([[400.19866943, 798.18634033],
       [403.87683105, 798.28338623],
       [400.38867188, 798.19830322],
       ...,
       [532.42120361, 731.26940918],
       [532.0680542 , 731.51141357],
       [531.77392578, 731.56878662]])

In [49]:
angles['L_Ankle_Pos'][:,1]

array([759.73651123, 759.82739258, 759.7479248 , 759.69885254,
       759.58032227, 759.56072998, 758.22369385, 758.09814453,
       757.92565918, 757.67285156, 757.77099609, 761.43621826,
       757.45959473, 757.36529541, 760.85644531, 757.26867676,
       760.82727051, 761.08441162, 760.8034668 , 759.29870605,
       759.57189941, 760.40771484, 759.99108887, 763.70117188,
       763.41680908, 759.64929199, 760.15423584, 759.95373535,
       759.97180176, 760.10003662, 758.0904541 , 761.89904785,
       761.80303955, 760.64038086, 761.81939697, 760.1472168 ,
       760.35528564, 760.25695801, 760.52130127, 760.11218262,
       761.86828613, 760.91790771, 761.05432129, 760.87854004,
       760.99005127, 760.78479004, 760.84625244, 760.92053223,
       760.86712646, 761.02258301, 760.94915771, 760.96411133,
       760.93768311, 760.84295654, 761.05596924, 759.29730225,
       758.57403564, 762.04119873, 758.23248291, 758.33312988,
       762.38085938, 758.67340088, 762.37701416, 761.59

In [56]:
metric = 'L_Shoulder_Pos'
fig = px.scatter(x=filtered_data['raw'][metric][:,0],y=filtered_data['raw'][metric][:,1]*-1)
fig.add_trace(go.Scatter(x=filtered_data['filtered'][metric][:,0],y=filtered_data['filtered'][metric][:,1]*-1,mode='markers+lines'))
fig.show()

In [55]:
metric = 'R_Elbow_Pos'
fig = px.scatter(x=filtered_data['raw'][metric][:,0],y=filtered_data['raw'][metric][:,1]*-1)
fig.add_trace(go.Scatter(x=filtered_data['filtered'][metric][:,0],y=filtered_data['filtered'][metric][:,1]*-1,mode='markers+lines'))
fig.show()

KeyError: 'R_Elbow_Pos'

In [76]:
fig = px.scatter(y=angles['L_1stToe_Pos'][:,1])
fig.show()

In [78]:
fig = px.scatter(df, y=["L_Hip_Angle",'R_Hip_Angle'],)
fig.update_layout(yaxis={'title':'Joint Angle (Degrees)'},xaxis={'title':'Frames'})
fig.show()

In [ ]:
fig = px.scatter(df, y=["L_Knee_Angle",'R_Knee_Angle'],)
fig.update_layout(yaxis={'title':'Joint Angle (Degrees)'},xaxis={'title':'Frames'})
fig.show()

In [12]:
fig = px.scatter(df, y=["left_ankle",'right_ankle'],)
fig.show()

In [29]:
l_shoulder = data['instance_info'][0]['instances'][0]['keypoints'][5]
l_hip = data['instance_info'][0]['instances'][0]['keypoints'][11]
l_knee = data['instance_info'][0]['instances'][0]['keypoints'][13]
l_ankle = data['instance_info'][0]['instances'][0]['keypoints'][15]
l_foot = data['instance_info'][0]['instances'][0]['keypoints'][19]

In [ ]:
python demo/topdown_demo_with_mmdet.py demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person.py https://download.openmmlab.com/mmpose/v1/projects/rtmpose/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth configs/wholebody_2d_keypoint/topdown_heatmap/coco-wholebody/td-hm_hrnet-w48_8xb32-210e_coco-wholebody-384x288.py https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_384x288_dark-f5726563_20200918.pth --input data/Riley_Taylor_20250609.mp4 --output-root vis_results --save-predictions --show

110.83613860803064

In [ ]:
python demo/topdown_demo_with_mmdet.py \
demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person.py `
https://download.openmmlab.com/mmpose/v1/projects/rtmpose/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth `
configs/wholebody_2d_keypoint/topdown_heatmap/coco-wholebody/td-hm_hrnet-w48_dark-8xb32-210e_coco-wholebody-384x288.py `
https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_384x288_dark-f5726563_20200918.pth `
--input data/Riley_Taylor_20250609.mp4 `
--output-root vis_results `
--show

In [ ]:
python demo/topdown_demo_with_mmdet.py demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person.py https://download.openmmlab.com/mmpose/v1/projects/rtmpose/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth configs/wholebody_2d_keypoint/topdown_heatmap/coco-wholebody/td-hm_hrnet-w48_8xb32-210e_coco-wholebody-384x288.py https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_wholebody_384x288_dark-f5726563_20200918.pth --input data/sFB_p1_m10505_xStart_d24062025_tb2.mp4 --output-root vis_results --save-predictions --show